# Lab 2-3: KServe 배포 (Jupyter Notebook)

## 📋 실습 개요

| 항목 | 내용 |
|------|------|
| **소요시간** | 40분 |
| **난이도** | ⭐⭐⭐ |
| **목표** | KServe로 프로덕션 모델 서빙 |

## 🎯 학습 목표

- KServe InferenceService 이해
- Python으로 Kubernetes 리소스 생성
- S3에서 모델 로드
- REST API를 통한 추론

---

© 2025 현대오토에버 MLOps Training

---

## 1. 환경 설정

### 1.1 패키지 설치

In [ ]:
# 필요한 패키지 설치
!pip install kubernetes requests boto3 mlflow -q

print("✅ 패키지 설치 완료!")

### 1.2 라이브러리 임포트

In [ ]:
from kubernetes import client, config
from kubernetes.client.rest import ApiException
import requests
import json
import time
import os
import base64

print("✅ 라이브러리 임포트 완료!")

### 1.3 Kubernetes 클라이언트 연결

In [ ]:
# ======================================================================
#                     Kubernetes 연결 설정
# ======================================================================

def connect_kubernetes():
    """
    Kubernetes 클러스터에 연결합니다.
    - Kubeflow 내부: load_incluster_config() 사용
    - 로컬: load_kube_config() 사용
    """
    try:
        # Kubeflow 내부 (Kubernetes Pod 내부)
        config.load_incluster_config()
        print("🏢 환경: Kubeflow Notebook (Kubernetes 클러스터 내부)")
        return "incluster"
    except config.ConfigException:
        # 로컬 환경
        config.load_kube_config()
        print("💻 환경: 로컬 (kubeconfig 사용)")
        return "local"

# 연결 실행
env_type = connect_kubernetes()

# API 클라이언트 생성
core_v1 = client.CoreV1Api()
custom_api = client.CustomObjectsApi()

# 연결 테스트
try:
    namespaces = core_v1.list_namespace()
    print(f"✅ Kubernetes 연결 성공! ({len(namespaces.items)}개 네임스페이스 발견)")
except Exception as e:
    print(f"❌ 연결 실패: {e}")

### 1.4 사용자 설정

⚠️ **중요**: 아래 설정을 본인의 환경에 맞게 수정하세요!

In [ ]:
# ======================================================================
#                     ⚠️ 사용자 설정 (수정 필요!)
# ======================================================================

# 수강생 번호 (01 ~ 30)
USER_NUM = "01"  # ← 본인 번호로 변경!

# AWS 자격증명 (강사 안내 참조)
AWS_ACCESS_KEY_ID = "YOUR_ACCESS_KEY"       # ← 수정 필요!
AWS_SECRET_ACCESS_KEY = "YOUR_SECRET_KEY"   # ← 수정 필요!
AWS_REGION = "ap-northeast-2"

# MLflow Run ID (Lab 2-2에서 확인)
MLFLOW_RUN_ID = "YOUR-RUN-ID"  # ← MLflow UI에서 복사!

# ======================================================================
#                     자동 설정 (수정 불필요)
# ======================================================================

NAMESPACE = f"kubeflow-user{USER_NUM}"
MODEL_NAME = "california-model"
S3_BUCKET = f"mlops-training-user{USER_NUM}"
MODEL_URI = f"s3://{S3_BUCKET}/mlflow-artifacts/1/{MLFLOW_RUN_ID}/artifacts/model"

print("="*60)
print("  사용자 설정 확인")
print("="*60)
print(f"  👤 사용자: user{USER_NUM}")
print(f"  📦 네임스페이스: {NAMESPACE}")
print(f"  🤖 모델 이름: {MODEL_NAME}")
print(f"  🪣 S3 버킷: {S3_BUCKET}")
print(f"  📍 모델 경로: {MODEL_URI}")
print("="*60)

---

## 2. AWS 자격증명 설정

KServe가 S3에서 모델을 로드하기 위해 AWS 자격증명이 필요합니다.

- **Secret**: AWS Access Key, Secret Key 저장
- **ConfigMap**: S3 엔드포인트, 리전 설정

### 2.1 AWS Secret 생성

In [ ]:
# ======================================================================
#  Step 2.1: AWS Secret 생성 (setup_credentials.sh 대체)
# ======================================================================

def create_aws_secret(namespace, access_key, secret_key, region):
    """
    AWS 자격증명 Secret을 생성합니다.
    
    기존 setup_credentials.sh의 다음 명령을 대체합니다:
    kubectl create secret generic aws-s3-credentials ...
    """
    secret_name = "aws-s3-credentials"
    
    # Secret 정의
    secret = client.V1Secret(
        api_version="v1",
        kind="Secret",
        metadata=client.V1ObjectMeta(
            name=secret_name,
            namespace=namespace
        ),
        type="Opaque",
        string_data={
            "AWS_ACCESS_KEY_ID": access_key,
            "AWS_SECRET_ACCESS_KEY": secret_key,
            "AWS_DEFAULT_REGION": region
        }
    )
    
    try:
        # 기존 Secret 삭제 (있으면)
        core_v1.delete_namespaced_secret(secret_name, namespace)
        print(f"  🗑️  기존 Secret '{secret_name}' 삭제")
    except ApiException as e:
        if e.status != 404:
            raise
    
    # 새 Secret 생성
    core_v1.create_namespaced_secret(namespace, secret)
    print(f"  ✅ Secret '{secret_name}' 생성 완료!")
    
    return secret_name

print("="*60)
print("  AWS Secret 생성")
print("="*60)

# Secret 생성 실행
secret_name = create_aws_secret(
    namespace=NAMESPACE,
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    region=AWS_REGION
)

### 2.2 S3 ConfigMap 생성

In [ ]:
# ======================================================================
#  Step 2.2: S3 ConfigMap 생성
# ======================================================================

def create_s3_configmap(namespace):
    """
    S3 설정 ConfigMap을 생성합니다.
    
    기존 setup_credentials.sh의 다음 명령을 대체합니다:
    kubectl create configmap s3-config ...
    """
    configmap_name = "s3-config"
    
    # ConfigMap 정의
    configmap = client.V1ConfigMap(
        api_version="v1",
        kind="ConfigMap",
        metadata=client.V1ObjectMeta(
            name=configmap_name,
            namespace=namespace
        ),
        data={
            "S3_ENDPOINT": "s3.amazonaws.com",
            "S3_USE_HTTPS": "1",
            "AWS_REGION": "ap-northeast-2"
        }
    )
    
    try:
        # 기존 ConfigMap 삭제 (있으면)
        core_v1.delete_namespaced_config_map(configmap_name, namespace)
        print(f"  🗑️  기존 ConfigMap '{configmap_name}' 삭제")
    except ApiException as e:
        if e.status != 404:
            raise
    
    # 새 ConfigMap 생성
    core_v1.create_namespaced_config_map(namespace, configmap)
    print(f"  ✅ ConfigMap '{configmap_name}' 생성 완료!")
    
    return configmap_name

print("="*60)
print("  S3 ConfigMap 생성")
print("="*60)

# ConfigMap 생성 실행
configmap_name = create_s3_configmap(NAMESPACE)

### 2.3 자격증명 확인

In [ ]:
# ======================================================================
#  Step 2.3: 자격증명 확인
# ======================================================================

print("="*60)
print("  자격증명 확인")
print("="*60)

# Secret 확인
try:
    secret = core_v1.read_namespaced_secret("aws-s3-credentials", NAMESPACE)
    print(f"  ✅ Secret 'aws-s3-credentials' 존재")
    print(f"     - Keys: {list(secret.data.keys())}")
except ApiException:
    print(f"  ❌ Secret 'aws-s3-credentials' 없음")

# ConfigMap 확인
try:
    cm = core_v1.read_namespaced_config_map("s3-config", NAMESPACE)
    print(f"  ✅ ConfigMap 's3-config' 존재")
    print(f"     - Keys: {list(cm.data.keys())}")
except ApiException:
    print(f"  ❌ ConfigMap 's3-config' 없음")

print("="*60)

---

## 3. InferenceService 배포

KServe InferenceService를 생성하여 모델을 배포합니다.

### 3.1 InferenceService 정의

In [ ]:
# ======================================================================
#  Step 3.1: InferenceService 정의 (manifests/inferenceservice.yaml 대체)
# ======================================================================

def create_inferenceservice_spec(model_name, namespace, storage_uri):
    """
    KServe InferenceService 스펙을 생성합니다.
    
    기존 manifests/inferenceservice.yaml을 Python dict로 대체합니다.
    """
    inference_service = {
        "apiVersion": "serving.kserve.io/v1beta1",
        "kind": "InferenceService",
        "metadata": {
            "name": model_name,
            "namespace": namespace,
            "annotations": {
                "serving.kserve.io/s3-endpoint": "s3.amazonaws.com",
                "serving.kserve.io/s3-region": "ap-northeast-2",
                "serving.kserve.io/s3-usehttps": "1",
                "serving.kserve.io/s3-useanoncredential": "false"
            }
        },
        "spec": {
            "predictor": {
                "serviceAccountName": "default",
                "model": {
                    "modelFormat": {
                        "name": "sklearn"
                    },
                    "storageUri": storage_uri,
                    "resources": {
                        "requests": {
                            "memory": "512Mi",
                            "cpu": "500m"
                        },
                        "limits": {
                            "memory": "1Gi",
                            "cpu": "1000m"
                        }
                    },
                    "env": [
                        {
                            "name": "AWS_ACCESS_KEY_ID",
                            "valueFrom": {
                                "secretKeyRef": {
                                    "name": "aws-s3-credentials",
                                    "key": "AWS_ACCESS_KEY_ID"
                                }
                            }
                        },
                        {
                            "name": "AWS_SECRET_ACCESS_KEY",
                            "valueFrom": {
                                "secretKeyRef": {
                                    "name": "aws-s3-credentials",
                                    "key": "AWS_SECRET_ACCESS_KEY"
                                }
                            }
                        },
                        {
                            "name": "AWS_DEFAULT_REGION",
                            "valueFrom": {
                                "secretKeyRef": {
                                    "name": "aws-s3-credentials",
                                    "key": "AWS_DEFAULT_REGION"
                                }
                            }
                        }
                    ]
                }
            }
        }
    }
    
    return inference_service

# InferenceService 스펙 생성
isvc_spec = create_inferenceservice_spec(MODEL_NAME, NAMESPACE, MODEL_URI)

print("="*60)
print("  InferenceService 스펙")
print("="*60)
print(json.dumps(isvc_spec, indent=2))

### 3.2 InferenceService 배포

In [ ]:
# ======================================================================
#  Step 3.2: InferenceService 배포 (deploy_kserve.sh 대체)
# ======================================================================

def deploy_inferenceservice(isvc_spec, namespace):
    """
    KServe InferenceService를 배포합니다.
    
    기존 deploy_kserve.sh의 다음 명령을 대체합니다:
    kubectl apply -f manifests/inferenceservice.yaml
    """
    model_name = isvc_spec["metadata"]["name"]
    
    print(f"  📦 InferenceService '{model_name}' 배포 중...")
    
    # 기존 InferenceService 삭제 (있으면)
    try:
        custom_api.delete_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        print(f"  🗑️  기존 InferenceService '{model_name}' 삭제")
        time.sleep(5)  # 삭제 완료 대기
    except ApiException as e:
        if e.status != 404:
            raise
    
    # 새 InferenceService 생성
    custom_api.create_namespaced_custom_object(
        group="serving.kserve.io",
        version="v1beta1",
        namespace=namespace,
        plural="inferenceservices",
        body=isvc_spec
    )
    
    print(f"  ✅ InferenceService '{model_name}' 생성 완료!")
    return model_name

print("="*60)
print("  InferenceService 배포")
print("="*60)

# 배포 실행
deploy_inferenceservice(isvc_spec, NAMESPACE)

### 3.3 배포 상태 모니터링

In [ ]:
# ======================================================================
#  Step 3.3: 배포 상태 모니터링
# ======================================================================

def wait_for_inferenceservice(model_name, namespace, timeout=300):
    """
    InferenceService가 Ready 상태가 될 때까지 대기합니다.
    
    기존 deploy_kserve.sh의 다음 명령을 대체합니다:
    kubectl wait --for=condition=Ready inferenceservice/california-model ...
    """
    print(f"  ⏳ InferenceService '{model_name}' Ready 대기 중...")
    print(f"     (최대 {timeout}초, 보통 2-3분 소요)")
    print()
    
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        try:
            isvc = custom_api.get_namespaced_custom_object(
                group="serving.kserve.io",
                version="v1beta1",
                namespace=namespace,
                plural="inferenceservices",
                name=model_name
            )
            
            # 상태 확인
            conditions = isvc.get("status", {}).get("conditions", [])
            ready_condition = next(
                (c for c in conditions if c.get("type") == "Ready"), 
                None
            )
            
            elapsed = int(time.time() - start_time)
            
            if ready_condition:
                status = ready_condition.get("status", "Unknown")
                reason = ready_condition.get("reason", "")
                
                if status == "True":
                    print(f"  ✅ InferenceService READY! ({elapsed}초 소요)")
                    return True
                else:
                    print(f"  ⏳ Status: {status} | Reason: {reason} ({elapsed}초)")
            else:
                print(f"  ⏳ 상태 확인 중... ({elapsed}초)")
            
        except ApiException as e:
            print(f"  ⚠️  조회 실패: {e.reason}")
        
        time.sleep(10)
    
    print(f"  ❌ 타임아웃! ({timeout}초 초과)")
    return False

print("="*60)
print("  배포 상태 모니터링")
print("="*60)

# Ready 상태 대기
is_ready = wait_for_inferenceservice(MODEL_NAME, NAMESPACE, timeout=300)

### 3.4 InferenceService 상태 확인

In [ ]:
# ======================================================================
#  Step 3.4: InferenceService 상태 확인
# ======================================================================

def get_inferenceservice_status(model_name, namespace):
    """
    InferenceService 상태를 조회합니다.
    
    기존 deploy_kserve.sh의 다음 명령을 대체합니다:
    kubectl get inferenceservice -n $NAMESPACE
    """
    try:
        isvc = custom_api.get_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        
        # 상태 정보 추출
        status = isvc.get("status", {})
        conditions = status.get("conditions", [])
        url = status.get("url", "N/A")
        address = status.get("address", {}).get("url", "N/A")
        
        print(f"  📊 InferenceService 상태")
        print(f"  " + "-"*50)
        print(f"  Name: {model_name}")
        print(f"  Namespace: {namespace}")
        print()
        print(f"  Conditions:")
        for cond in conditions:
            ctype = cond.get("type", "Unknown")
            cstatus = cond.get("status", "Unknown")
            symbol = "✅" if cstatus == "True" else "⏳" if cstatus == "Unknown" else "❌"
            print(f"    {symbol} {ctype}: {cstatus}")
        
        print()
        print(f"  URLs:")
        print(f"    External: {url}")
        print(f"    Internal: {address}")
        
        return isvc
        
    except ApiException as e:
        print(f"  ❌ 조회 실패: {e.reason}")
        return None

print("="*60)
print("  InferenceService 상태 확인")
print("="*60)

isvc_status = get_inferenceservice_status(MODEL_NAME, NAMESPACE)

### 3.5 Pod 상태 확인

In [ ]:
# ======================================================================
#  Step 3.5: Pod 상태 확인
# ======================================================================

def get_model_pods(model_name, namespace):
    """
    모델 서빙 Pod 목록을 조회합니다.
    
    기존 deploy_kserve.sh의 다음 명령을 대체합니다:
    kubectl get pods -n $NAMESPACE -l serving.kserve.io/inferenceservice=$MODEL_NAME
    """
    label_selector = f"serving.kserve.io/inferenceservice={model_name}"
    
    pods = core_v1.list_namespaced_pod(
        namespace=namespace,
        label_selector=label_selector
    )
    
    print(f"  📦 모델 서빙 Pod 목록")
    print(f"  " + "-"*50)
    
    if not pods.items:
        print(f"  ⚠️  Pod를 찾을 수 없습니다.")
        return None
    
    for pod in pods.items:
        name = pod.metadata.name
        phase = pod.status.phase
        symbol = "✅" if phase == "Running" else "⏳" if phase == "Pending" else "❌"
        
        # Container 상태
        containers = []
        for cs in (pod.status.container_statuses or []):
            ready = "Ready" if cs.ready else "NotReady"
            containers.append(f"{cs.name}:{ready}")
        
        print(f"  {symbol} {name}")
        print(f"     Phase: {phase}")
        print(f"     Containers: {', '.join(containers) if containers else 'N/A'}")
    
    return pods.items[0] if pods.items else None

print("="*60)
print("  Pod 상태 확인")
print("="*60)

model_pod = get_model_pods(MODEL_NAME, NAMESPACE)

---

## 4. 추론 테스트

배포된 모델에 REST API로 추론 요청을 보냅니다.

### 4.1 테스트 데이터 준비

In [ ]:
# ======================================================================
#  Step 4.1: 테스트 데이터 준비
# ======================================================================

# California Housing 데이터셋 피처 (8개)
# [MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude]

test_data = {
    "instances": [
        [8.3252, 41.0, 6.984, 1.024, 322.0, 2.556, 37.88, -122.23],  # 샘플 1
        [5.6431, 52.0, 5.817, 1.073, 558.0, 2.547, 37.85, -122.25],  # 샘플 2
        [3.8462, 35.0, 6.238, 1.058, 1212.0, 2.689, 37.87, -122.24]  # 샘플 3
    ]
}

print("="*60)
print("  테스트 데이터")
print("="*60)
print(f"  피처: [MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Lat, Lon]")
print()
for i, instance in enumerate(test_data["instances"]):
    print(f"  샘플 {i+1}: {instance}")

### 4.2 클러스터 내부 추론 테스트 (Kubeflow 환경)

In [ ]:
# ======================================================================
#  Step 4.2: 클러스터 내부 추론 테스트
# ======================================================================

def test_inference_internal(model_name, namespace, test_data):
    """
    클러스터 내부에서 추론 테스트를 수행합니다.
    Kubeflow Notebook 환경에서 실행 시 사용합니다.
    """
    # 클러스터 내부 서비스 URL
    url = f"http://{model_name}.{namespace}.svc.cluster.local/v1/models/{model_name}:predict"
    
    print(f"  🌐 추론 URL: {url}")
    print()
    
    try:
        response = requests.post(
            url,
            json=test_data,
            headers={"Content-Type": "application/json"},
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"  ✅ 추론 성공!")
            print()
            print(f"  📊 결과:")
            predictions = result.get("predictions", [])
            for i, pred in enumerate(predictions):
                print(f"    샘플 {i+1}: ${pred*100000:,.0f} (예측 주택 가격)")
            return result
        else:
            print(f"  ❌ 추론 실패: HTTP {response.status_code}")
            print(f"     {response.text}")
            return None
            
    except requests.exceptions.ConnectionError:
        print(f"  ⚠️  연결 실패 (클러스터 외부 환경일 수 있음)")
        print(f"     → 다음 셀의 '포트 포워딩' 방법을 사용하세요.")
        return None
    except Exception as e:
        print(f"  ❌ 오류: {e}")
        return None

print("="*60)
print("  클러스터 내부 추론 테스트")
print("="*60)

# 클러스터 내부 테스트 실행
if env_type == "incluster":
    result = test_inference_internal(MODEL_NAME, NAMESPACE, test_data)
else:
    print("  ℹ️  로컬 환경에서는 포트 포워딩을 사용하세요.")
    print("     → 다음 셀 실행")

### 4.3 포트 포워딩 방식 추론 테스트 (로컬 환경)

⚠️ 로컬 환경에서는 아래 명령어를 **터미널에서 별도로 실행**해야 합니다:

```bash
# 터미널에서 실행 (이 셀에서 실행하지 마세요!)
kubectl port-forward -n kubeflow-user01 svc/california-model-predictor 8081:80
```

In [ ]:
# ======================================================================
#  Step 4.3: 포트 포워딩 방식 추론 테스트 (로컬 환경)
# ======================================================================

def test_inference_portforward(model_name, test_data, port=8081):
    """
    포트 포워딩을 통해 추론 테스트를 수행합니다.
    로컬 환경에서 실행 시 사용합니다.
    
    기존 test_inference.sh를 대체합니다.
    
    사전 요구사항:
    터미널에서 다음 명령 실행:
    kubectl port-forward -n kubeflow-user01 svc/california-model-predictor 8081:80
    """
    url = f"http://localhost:{port}/v1/models/{model_name}:predict"
    
    print(f"  🌐 추론 URL: {url}")
    print()
    
    try:
        response = requests.post(
            url,
            json=test_data,
            headers={"Content-Type": "application/json"},
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"  ✅ 추론 성공!")
            print()
            print(f"  📊 결과:")
            predictions = result.get("predictions", [])
            for i, pred in enumerate(predictions):
                print(f"    샘플 {i+1}: ${pred*100000:,.0f} (예측 주택 가격)")
            return result
        else:
            print(f"  ❌ 추론 실패: HTTP {response.status_code}")
            print(f"     {response.text}")
            return None
            
    except requests.exceptions.ConnectionError:
        print(f"  ❌ 연결 실패!")
        print()
        print(f"  💡 해결 방법:")
        print(f"     터미널에서 다음 명령을 먼저 실행하세요:")
        print(f"     kubectl port-forward -n {NAMESPACE} svc/{MODEL_NAME}-predictor {port}:80")
        return None
    except Exception as e:
        print(f"  ❌ 오류: {e}")
        return None

print("="*60)
print("  포트 포워딩 방식 추론 테스트")
print("="*60)

if env_type == "local":
    result = test_inference_portforward(MODEL_NAME, test_data, port=8081)
else:
    print("  ℹ️  Kubeflow 환경에서는 이전 셀의 결과를 확인하세요.")

---

## 5. 정리 (선택사항)

실습 후 리소스를 정리합니다.

In [ ]:
# ======================================================================
#  Step 5: 리소스 정리 (선택사항)
# ======================================================================

def cleanup_resources(model_name, namespace, delete_credentials=False):
    """
    실습 리소스를 정리합니다.
    """
    print("  🗑️  리소스 정리 중...")
    
    # InferenceService 삭제
    try:
        custom_api.delete_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        print(f"  ✅ InferenceService '{model_name}' 삭제")
    except ApiException as e:
        if e.status == 404:
            print(f"  ℹ️  InferenceService '{model_name}' 이미 없음")
        else:
            print(f"  ⚠️  InferenceService 삭제 실패: {e.reason}")
    
    if delete_credentials:
        # Secret 삭제
        try:
            core_v1.delete_namespaced_secret("aws-s3-credentials", namespace)
            print(f"  ✅ Secret 'aws-s3-credentials' 삭제")
        except ApiException:
            pass
        
        # ConfigMap 삭제
        try:
            core_v1.delete_namespaced_config_map("s3-config", namespace)
            print(f"  ✅ ConfigMap 's3-config' 삭제")
        except ApiException:
            pass
    
    print("  ✅ 정리 완료!")

# ⚠️ 아래 주석을 해제하면 리소스가 삭제됩니다!
# print("="*60)
# print("  리소스 정리")
# print("="*60)
# cleanup_resources(MODEL_NAME, NAMESPACE, delete_credentials=False)

print("💡 리소스 정리가 필요하면 위 셀의 주석을 해제하고 실행하세요.")

---

## ✅ 실습 완료!

### 학습한 내용

- ✅ Kubernetes Python 라이브러리로 Secret/ConfigMap 생성
- ✅ KServe InferenceService 배포
- ✅ 배포 상태 모니터링
- ✅ REST API를 통한 추론 테스트

### 📚 다음 단계

1. **Lab 3-1**: Prometheus/Grafana 모니터링
2. **Lab 3-2**: CI/CD 파이프라인 통합
3. **Project**: E2E MLOps 파이프라인 구축

---

© 2025 현대오토에버 MLOps Training - Lab 2-3  
**Environment**: Kubeflow Jupyter Notebook  
**Model Serving**: KServe InferenceService